In [26]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [27]:
df = pd.read_excel("learning.xlsx",index_col = 0)
print(len(df))
df["Title"] = df["Title"].astype(str)
df["Description"] = df["Description"].astype(str)
df["StrPrimaryCategoryID"] = df["PrimaryCategoryID"].apply(lambda x: "category"+str(x))
df['Title'] = df[['StrPrimaryCategoryID','Title']].apply(lambda x: ' '.join(x), axis=1)
df['Content'] = df[['Title', 'Description','StrPrimaryCategoryID']].apply(lambda x: ' '.join(x), axis=1)
df.head()

4331


,ItemID,value,_currencyID,ViewItemURLForNaturalSearch,Title,PrimaryCategoryID,Country,Description,group,StrPrimaryCategoryID,Content
0,132529667929,164.77,USD,https://www.ebay.com/itm/Philips-ATL-Dio-Stift...,category71927 Philips ATL Dio Stiftsonde CW Do...,71927,DE,Philips ATL Dio Stiftsonde CW 3.2 MM ...,MechProbe,category71927,category71927 Philips ATL Dio Stiftsonde CW Do...
1,112314171710,326.79,USD,https://www.ebay.com/itm/Kretz-Endovaginal-5-0...,category71927 Kretz Endovaginal 5.0 MHz E.V 4...,71927,DE,"Kretz Endovaginalsonde 5.0 MHz, Typ: E. V 48 H...",Array,category71927,category71927 Kretz Endovaginal 5.0 MHz E.V 4...
2,372840599500,24.92,USD,https://www.ebay.com/itm/MOSKVICH-21412-1-6-80...,category63711 FOR MOSKVICH 21412 1.6 80HP -97 ...,63711,GB,A to Z Motor Spares (Wakefield) Step1. C...,NotUS,category63711,category63711 FOR MOSKVICH 21412 1.6 80HP -97 ...
3,324091756506,1745.82,USD,https://www.ebay.com/itm/GE-546L-Ultrasound-Pr...,category184515 GE 546L Ultrasound Probe / Tran...,184515,US,546L transducerCompatible SystemsGE 546L line...,Array,category184515,category184515 GE 546L Ultrasound Probe / Tran...
4,173300186560,19.25,USD,https://www.ebay.com/itm/Comline-Front-Right-S...,category33592 Comline Front Right Stabiliser L...,33592,GB,Comline Stabiliser Link BarIn order to check t...,NotUS,category33592,category33592 Comline Front Right Stabiliser L...


In [28]:
df['Title_Clean'] = df['Title'].str.replace("\r", " ")
df['Title_Clean'] = df['Title_Clean'].str.replace("\n", " ")
df['Title_Clean'] = df['Title_Clean'].str.replace("    ", " ")
df['Title_Clean'] = df['Title_Clean'].str.replace('"', '')
df['Title_Clean'] = df['Title_Clean'].str.lower()

punctuation_signs = list('?:!,;-/*"')

for punct_sign in punctuation_signs:
    df['Title_Clean'] = df['Title_Clean'].str.replace(punct_sign, '')
    
df['Title_Clean'] = df['Title_Clean'].str.replace("'s", "")

In [29]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

------------------------------------------------------------


[nltk_data] Downloading package punkt to /home/kelu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kelu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Title_Clean']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [31]:
df['Title_Clean'] = lemmatized_text_list
df['Title_Clean']

0       category71927 philips atl dio stiftsonde cw do...
1       category71927 kretz endovaginal 5.0 mhz  e.v 4...
2       category63711 for moskvich 21412 1.6 80hp 97 n...
3       category184515 ge 546l ultrasound probe  trans...
4       category33592 comline front right stabiliser l...
                              ...                        
4326    category184542 atl c84v ivt ultrasound transdu...
4327    category33583 meyle koppelstange va beidseitig...
4328    category61523 bmw f45 f46 f48 f15 g30 g31 g11 ...
4329    category184542 aloka ssd900 portable ultrasoun...
4330    category57357 delphi bremsbelagsatz scheibenbr...
Name: Title_Clean, Length: 4331, dtype: object

In [32]:
# Downloading the stop words list
nltk.download('stopwords')
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/kelu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
from tqdm.notebook import trange, tqdm

for stop_word in tqdm(stop_words, 
                         total=len(stop_words), 
                         desc="Processing records"):
    regex_stopword = r"\b" + stop_word + r"\b"
    df['Title_Clean'] = df['Title_Clean'].str.replace(regex_stopword, '')

In [34]:
category_codes = {}
category_names = {}
i = 0
for k in df.group.unique().tolist():
    category_codes[k] = i
    category_names[i] = k
    i += 1

In [35]:
df['Category_Code'] = df['group']
df = df.replace({'Category_Code':category_codes})
df.head()

,ItemID,value,_currencyID,ViewItemURLForNaturalSearch,Title,PrimaryCategoryID,Country,Description,group,StrPrimaryCategoryID,Content,Title_Clean,Category_Code
0,132529667929,164.77,USD,https://www.ebay.com/itm/Philips-ATL-Dio-Stift...,category71927 Philips ATL Dio Stiftsonde CW Do...,71927,DE,Philips ATL Dio Stiftsonde CW 3.2 MM ...,MechProbe,category71927,category71927 Philips ATL Dio Stiftsonde CW Do...,category71927 philips atl dio stiftsonde cw do...,0
1,112314171710,326.79,USD,https://www.ebay.com/itm/Kretz-Endovaginal-5-0...,category71927 Kretz Endovaginal 5.0 MHz E.V 4...,71927,DE,"Kretz Endovaginalsonde 5.0 MHz, Typ: E. V 48 H...",Array,category71927,category71927 Kretz Endovaginal 5.0 MHz E.V 4...,category71927 kretz endovaginal 5.0 mhz e.v 4...,1
2,372840599500,24.92,USD,https://www.ebay.com/itm/MOSKVICH-21412-1-6-80...,category63711 FOR MOSKVICH 21412 1.6 80HP -97 ...,63711,GB,A to Z Motor Spares (Wakefield) Step1. C...,NotUS,category63711,category63711 FOR MOSKVICH 21412 1.6 80HP -97 ...,category63711 moskvich 21412 1.6 80hp 97 new ...,2
3,324091756506,1745.82,USD,https://www.ebay.com/itm/GE-546L-Ultrasound-Pr...,category184515 GE 546L Ultrasound Probe / Tran...,184515,US,546L transducerCompatible SystemsGE 546L line...,Array,category184515,category184515 GE 546L Ultrasound Probe / Tran...,category184515 ge 546l ultrasound probe trans...,1
4,173300186560,19.25,USD,https://www.ebay.com/itm/Comline-Front-Right-S...,category33592 Comline Front Right Stabiliser L...,33592,GB,Comline Stabiliser Link BarIn order to check t...,NotUS,category33592,category33592 Comline Front Right Stabiliser L...,category33592 comline front right stabiliser l...,2


# Going in the fun

In [36]:
DF = df[~(df.group == "None")] 
X_train, X_test, y_train, y_test = train_test_split(df['Title_Clean'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [37]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [38]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(3681, 300)
(650, 300)


In [39]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Array' category:
  . Most correlated unigrams:
. category71927
. ge
. category184515
. array
. transducer
  . Most correlated bigrams:
. ultrasound probe
. ultrasound transducer

# 'Boards' category:
  . Most correlated unigrams:
. 5000
. hdi
. 4000
. philipsatl
. board
  . Most correlated bigrams:
. hdi 4000
. 4000 ultrasound

# 'Cables' category:
  . Most correlated unigrams:
. new
. brand
. medical
. ce
. use
  . Most correlated bigrams:
. brand new
. transducer brand

# 'Doppler' category:
  . Most correlated unigrams:
. philips
. mhz
. 10
. doppler
. cw
  . Most correlated bigrams:
. category184515 philips
. philips atl

# 'FullMachine' category:
  . Most correlated unigrams:
. system
. rite
. site
. transducers
. ultraschallgerät
  . Most correlated bigrams:
. bard site
. ultrasound system

# 'HH' category:
  . Most correlated unigrams:
. machine
. vet
. rectal
. veterinary
. scanner
  . Most correlated bigrams:
. rectal probe
. ultrasound scanner

# 'MechProbe' category:
  . 

In [40]:
if 1:
    pathPickle = "Pickles"
else:
    pathPickle = "Pickles_FullDesc"
    
with open(pathPickle+'/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
with open(pathPickle+'/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open(pathPickle+'/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open(pathPickle+'/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open(pathPickle+'/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open(pathPickle+'/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
with open(pathPickle+'/DFNONE.pickle', 'wb') as output:
    pickle.dump(DFNONE, output)
    
# features_train
with open(pathPickle+'/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open(pathPickle+'/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open(pathPickle+'/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open(pathPickle+'/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open(pathPickle+'/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)
    
    
with open(pathPickle+'/category_codes.pickle', 'wb') as output:
    pickle.dump(category_codes, output)


In [41]:
DFNONE

,ItemID,value,_currencyID,ViewItemURLForNaturalSearch,Title,PrimaryCategoryID,Country,Description,group,StrPrimaryCategoryID,Content,Title_Clean,Category_Code
